# Tutorial HallThruster.jl

HallThruster is an efficient 1D fluid model to simulate Hall Thruster discharges. 
This tutorial will show you how to get started with HallThruster.jl and will compare our results to the Landmark study. Detailed information can be found in the [Documentation](https://um-pepl.github.io/HallThruster.jl/dev/).

First install HallThruster.jl

In [1]:
using Pkg
Pkg.add(url="https://github.com/UM-PEPL/HallThruster.jl")

In [ ]:
Pkg.add("Plots")

Once HallThruster is installed, let's load it along with Plots for visualizing results

In [10]:
using HallThruster
using Plots

ErrorException: Failed to precompile HallThruster [2311f341-5e6d-4941-9e3e-3ce0ae0d9ed6] to C:\Users\thoma\.julia\compiled\v1.7\HallThruster\jl_5B37.tmp.

Let's try and replicate the LANDMARK benchmarks. 

Let us define the following `run_sim()` as a helper function. The most important arguments are `end_time; ncells, nsave, dt`. When increasing the amount of cells, the timestep has to be reduced accordingly to satisfy the CFL condition. `nsave` specifies the amount of times the solution is saved in constant time intervals. 

Here we have the Landmark source terms and energy loss terms given as a standard, and we can select one of the 3 test cases to run. We initialize a new simulation and define the domain as twice the thruster length, as evident by the argument `L = 0.05`. The default flux is set to `rusanov` and the time marching algorithm from DifferentialEquations.jl for the heavy species is a strong stability presering Runge Kutta scheme of second order `SSPRK22`. 

In [11]:
function run_sim(end_time = 0.0002; ncells = 50, nsave = 2, dt = 1e-8,
        implicit_energy = 1.0, reconstruct = false, limiter = HallThruster.osher,
        restart_file = nothing, case = 1,
        alg = SSPRK22(stage_limiter! = HallThruster.stage_limiter!, step_limiter! = HallThruster.stage_limiter!),
        flux = HallThruster.rusanov, ionization_model = HallThruster.LandmarkIonizationLookup(), transition = HallThruster.LinearTransition(0.001, 0.0),
        electron_neutral_model = HallThruster.LandmarkElectronNeutral(), coupled = true, energy_equation = :LANDMARK,
        progress_interval = 0, WENO = false, L = 0.05
    )

    domain = (0.0, L)

    #Landmark cases loss frequencies
    αϵ_in, αϵ_out = if case == 1
        (1.0, 1.0)
    elseif case == 2
        (0.5, 1.0)
    elseif case == 3
        (0.4, 1.0)
    end

    scheme = HallThruster.HyperbolicScheme(flux, limiter, reconstruct, WENO)

    ϵ_anode = 3.0
    ϵ_cathode = 3.0

    config = HallThruster.Config(;
        ncharge = 1,
        anode_Te = 2/3 * ϵ_anode,
        cathode_Te = 2/3 * ϵ_cathode,
        discharge_voltage = 300.0,
        excitation_model = HallThruster.LandmarkExcitationLookup(),
        wall_loss_model = HallThruster.ConstantSheathPotential(-20, αϵ_in, αϵ_out),
        wall_collision_freq = 1.0 * 1e7,
        implicit_energy = implicit_energy,
        transition_function = transition,
        electron_pressure_coupled = coupled,
        progress_interval = progress_interval,
        neutral_velocity = 150.0,
        neutral_temperature = 0.0,
        ion_temperature = 0.0,
        thruster = HallThruster.SPT_100,
        anode_mass_flow_rate = 5e-6,
        scheme,
        electron_neutral_model,
        ionization_model,
        domain,
        energy_equation,
        WENO = WENO
    )

    @time sol = HallThruster.run_simulation(config, dt, end_time, ncells, nsave; restart_file, alg)
    return sol
end

     Cloning git-repo `https://github.com/UM-PEPL/HallThruster.jl`
    Updating git-repo `https://github.com/UM-PEPL/HallThruster.jl`
    Updating registry at `C:\Users\thoma\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
ERROR: LoadError: ArgumentError: Package HallThruster does not have SpecialFunctions in its dependencies:
- If you have HallThruster checked out for development and have
  added SpecialFunctions as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with HallThruster
Stacktrace:
 [1] require(into::Module, mod::Symbol)
   @ Base .\loading.jl:980
 [2] include
   @ .\Base.jl:418 [inlined]
 [3] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
   @ Base .\loading.jl:1318
 [4] t

run_sim (generic function with 2 methods)

Now we can use `run_sim()` to perform simulations. 

In [12]:
sol = run_sim(1e-3; ncells=200, nsave=1000, case = 1, dt = 0.7e-8);

UndefVarError: UndefVarError: HallThruster not defined

Can analyse solution, calculate currents, do postprocessing. Should add plotting functions to package?

In [13]:
HallThruster.plot(sol)

UndefVarError: UndefVarError: HallThruster not defined